<a href="https://colab.research.google.com/github/VMBoehm/N3AS_Project_Malika/blob/main/malika_Copy_of_NeutrinoClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#! ls ./drive/MyDrive/

In [3]:
import matplotlib.pyplot as plt

In [5]:
side_len = 128

In [6]:
train_gen = keras.preprocessing.image_dataset_from_directory(
    './drive/MyDrive/myz15',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(side_len, side_len),
    shuffle=True,
    seed=1534,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    follow_links=False
)

valid_gen = keras.preprocessing.image_dataset_from_directory(
    './drive/MyDrive/myz15',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(side_len, side_len),
    shuffle=True,
    seed=1534,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    follow_links=False
)

Found 15999 files belonging to 1 classes.
Using 12800 files for training.
Found 15999 files belonging to 1 classes.
Using 3199 files for validation.


In [1]:
plt.imshow(np.squeeze(next(iter(valid_gen))[0][0]/255))

NameError: ignored

In [8]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(side_len, side_len, 3),
    include_top=False) 

base_model.trainable = False

83697664/83683744 [==============================] - 1s 0us/step


In [17]:
#from cv2 import cv2

In [18]:
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

class Gray2VGGInput( Layer ) :
    """Custom conversion layer
    """
    def build( self, x ) :
        self.built = True
        return

    def call( self, x ) :
        rgb_x = K.concatenate( [x,x,x], axis=-1 )
        norm_x = rgb_x
        return norm_x

    def compute_output_shape( self, input_shape ) :
        return input_shape[:3] + (3,)

In [63]:
inputs          = keras.Input(shape=(side_len, side_len,1))
#vgg_input_image = Gray2VGGInput( name='gray_to_rgb_norm')(inputs)
input_con = tf.image.grayscale_to_rgb( ## this did not help but did give the same results 
    inputs,
    name=None
)
x               =  input_con #vgg_input_image ## maybe I can try tf.image.grayscale_to_rgb
scale_layer     = keras.layers.experimental.preprocessing.Rescaling(scale=1./255)

x = scale_layer(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(8, activation='relu')(x)
outputs = keras.layers.Dense(1)(x)
model   = keras.Model(inputs, outputs)


In [64]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

#CategoricalCrossentropy showed that binary acuraccy is not learning
# things I used BinaryFocalCrossentropy and Hinge and BinaryIoU

In [65]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 128, 128, 1)]     0         
                                                                 
 tf.image.grayscale_to_rgb_1  (None, 128, 128, 3)      0         
  (TFOpLambda)                                                   
                                                                 
 rescaling_12 (Rescaling)    (None, 128, 128, 3)       0         
                                                                 
 xception (Functional)       (None, 4, 4, 2048)        20861480  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 8)                 1639

In [66]:
# TASK: add validation data (https://keras.io/api/models/model_training_apis/)
model.fit(train_gen, epochs=10, validation_data=valid_gen)

Epoch 1/10
121/400 [========>.....................] - ETA: 28s - loss: 0.4974 - binary_accuracy: 1.0000

KeyboardInterrupt: ignored

In [20]:
model.predict_generator(train_gen, steps=4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


array([[ -5.826433 ],
       [ -4.3159246],
       [ -3.4278915],
       [ -3.6935027],
       [ -8.8251   ],
       [ -3.3105195],
       [ -4.6037893],
       [ -5.693119 ],
       [ -5.2991858],
       [ -6.6345997],
       [ -3.762495 ],
       [-10.017114 ],
       [ -2.3365877],
       [ -3.7319958],
       [ -2.8115683],
       [ -6.853443 ],
       [ -3.626202 ],
       [ -4.746039 ],
       [ -3.428169 ],
       [ -6.8850126],
       [ -5.6930246],
       [ -6.020764 ],
       [ -4.223012 ],
       [ -3.371087 ],
       [ -5.916847 ],
       [ -9.340217 ],
       [ -3.747883 ],
       [ -3.9128401],
       [ -4.137532 ],
       [ -2.8524075],
       [ -3.299277 ],
       [ -2.8520842],
       [ -7.018502 ],
       [ -5.7277827],
       [ -3.495673 ],
       [ -4.7647514],
       [ -4.327639 ],
       [ -4.333083 ],
       [ -3.4924774],
       [ -4.6585207],
       [ -6.9068127],
       [ -5.4481378],
       [ -4.405964 ],
       [ -4.652198 ],
       [ -4.5063543],
       [ -